In [2]:
pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk]

Note: you may need to restart the kernel to use updated packages.


In [223]:
pip install -U -q "google-genai"

Note: you may need to restart the kernel to use updated packages.


In [83]:
import vertexai
from vertexai import agent_engines
vertexai.init(
    project="content-creation-agent-468223",  # Your project ID.
    location="us-central1",                # Your cloud region.
    staging_bucket="gs://launchpad_marketing",  # Your staging bucket.
)

In [268]:
from google import genai
from google.genai.types import Content, CreateCachedContentConfig, HttpOptions, Part


def generate_blog_post(
    product_name: str,
    description: str,
    features: str,
    audience:str
):
    
    """
    Generates a product launch blog post using Vertex AI.

    The output includes a title, intro, body, and conclusion — based on the product brief.
    """

    model='gemini-2.0-flash'
    client = genai.Client(http_options=HttpOptions(api_version="v1"))

    # Increase the number of word range so that we allow the model to finish its sentence
    system_instruction = f'''
                            Write a well-structured product announcement blog post that is:
                            - At least 500 words long
                            - No more than 600 words.
                            
                            Content must include:
                            - A product name,
                            - A description,
                            - Key features,
                            - Target Audience

                            Format:
                            - Engaging and informative title
                            - Brief introduction (1 paragraph)
                            - Main body (2–3 paragraphs highlighting benefits/features)
                            - Conclusion (summary + call to action)
                            '''

    user_instruction = f"""
                        Product Name: {product_name}
                        Description: {description}
                        Key Features: {features}
                        Target Audience: {audience}
                        """

    response = client.models.generate_content(
        model=model,
        contents = user_instruction,
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
            max_output_tokens=900,
            temperature=0.6,
            response_logprobs=True,
            logprobs=3,
          ),
        )

    return response.text

In [269]:
blog_post_response = generate_blog_post(
    product_name = "SmartNotes",
    description = "An AI-powered note-taking app for meetings",
    features = "Real-time transcription, Smart summaries, Action item detection",
    audience = "Remote workers, Product managers, and Sales teams"
)

print(blog_post_response)
print(f"{len(blog_post_response.split())} words")

## Stop Drowning in Meeting Notes: Introducing SmartNotes, Your AI-Powered Meeting Assistant!

Are you tired of spending hours after meetings deciphering your scribbled notes? Do you struggle to remember key decisions and action items? Say goodbye to note-taking fatigue and hello to SmartNotes, the revolutionary AI-powered note-taking app designed to transform the way you approach meetings. SmartNotes leverages the power of artificial intelligence to provide real-time transcription, smart summaries, and automatic action item detection, freeing you to focus on what truly matters: participating and contributing to the conversation.

SmartNotes is more than just a recording app; it's your intelligent meeting assistant. Imagine a world where you can fully engage in discussions without the pressure of frantically scribbling notes. With its real-time transcription feature, SmartNotes converts spoken words into text as the meeting unfolds. No more missed points or garbled phrases! But the mag

In [271]:
from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp
from google.genai import types

model="gemini-2.0-flash"

#Set safety settings:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.6,
   max_output_tokens= 1000,
   top_p= 0.95,
)

#Define Agent
agent = Agent(
    model=model,
    name="Blog_Post_Agent",
    description=("Agent that writes a 400-500 word announcement blog post"),
    generate_content_config=generate_content_config,
    tools=[generate_blog_post],
)

#Create Agent using ADK
app = AdkApp(agent=agent)

In [272]:
#Test Agent
for event in app.stream_query(
    user_id="user_123",
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)

{'content': {'parts': [{'text': "Okay, I can help you write a blog post announcing VisionLink Pro. I'll use the information you provided to generate a draft.\n"}, {'function_call': {'id': 'adk-badb109a-7e25-49c6-bcfc-d79b447d816b', 'args': {'product_name': 'VisionLink Pro', 'audience': 'remote teams, global companies, and project managers', 'features': 'support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video', 'description': 'It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.'}, 'name': 'generate_blog_post'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 100, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 100}], 'prompt_token_count': 186, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 186}], 'total_token_count': 286, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-a91e1ed3-af58-4155-

In [273]:
def session_service_builder():
  from google.adk.sessions import InMemorySessionService

  return InMemorySessionService()

In [274]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(
   agent=agent,                                      # Required.
   session_service_builder=session_service_builder,  # Optional.
)

In [275]:
session = app.create_session(user_id="user_123")

In [276]:
app.list_sessions(user_id="user_123")

ListSessionsResponse(sessions=[Session(id='aa547b52-801b-4b8a-b843-9315dbf2fb15', app_name='default-app-name', user_id='user_123', state={}, events=[], last_update_time=1754620685.1987162)])

In [277]:
for event in app.stream_query(
    user_id="user_123",
    session_id="aa547b52-801b-4b8a-b843-9315dbf2fb15", # Optional. you can pass in the session_id when querying the agent
    message="Create me a blog post about a product called Eddy",
):
    print(event)

{'content': {'parts': [{'text': "Okay, I need some more information to write a good blog post.  Tell me:\n\n*   Who is the target audience for Eddy?\n*   What does Eddy do? (A brief description)\n*   What are Eddy's key features?"}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 53, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 53}], 'prompt_token_count': 109, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 109}], 'total_token_count': 162, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-b8790c83-11f4-43a7-bf9a-da394d1eedea', 'author': 'Blog_Post_Agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '1f234fa4-045d-4b31-ad9d-1bee9912c852', 'timestamp': 1754620883.328787}


In [278]:
#Test Agent
for event in app.stream_query(
    user_id="user_123",
    session_id="aa547b52-801b-4b8a-b843-9315dbf2fb15",
    message=""" It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)

{'content': {'parts': [{'text': 'OK, I have all the information I need. I will generate a blog post about Eddy.\n'}, {'function_call': {'id': 'adk-5cee620d-a7a0-4628-98b6-8667755eb338', 'args': {'audience': 'Remote teams, global companies, and project managers', 'description': 'AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration', 'features': 'Support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video', 'product_name': 'Eddy'}, 'name': 'generate_blog_post'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 84, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 84}], 'prompt_token_count': 235, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 235}], 'total_token_count': 319, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-59e713f0-ecea-4409-a696-0f14009509c3', 'author': 'Blog_Post_Agent', 'actions': {'state

In [285]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]","cloudpickle==3.1.1","pydantic==2.11.7"],
)

Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.107.0'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'cloudpickle==3.1.1', 'pydantic==2.11.7']
Using bucket launchpad_marketing
Wrote to gs://launchpad_marketing/agent_engine/agent_engine.pkl
Writing to gs://launchpad_marketing/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://launchpad_marketing/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/1014433624001/locations/us-central1/reasoningEngines/1641201424362635264/operations/6567433495881187328
View progress and logs at https://console.cloud.google.com/logs/query?project=content-creation-agent-468223
AgentEngine created. Resource name: projects/1014433624001/locations/us-central1/reasoningEngines/1641201424362635264
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.get(

In [288]:
from vertexai.preview.reasoning_engines import AdkApp

remote_agent = AdkApp(
   agent=remote_agent,                               # Required.
   session_service_builder=session_service_builder,  # Optional.
)

In [289]:
session = remote_agent.create_session(user_id="user_123")

In [293]:
remote_agent.list_sessions(user_id="user_123")

ListSessionsResponse(sessions=[Session(id='d239c214-778b-41ae-b7d9-95ae0a3907eb', app_name='default-app-name', user_id='user_123', state={}, events=[], last_update_time=1754622157.1974843)])

In [295]:
for event in remote_agent.stream_query(
    user_id="user_123",
    message="""I want to announce a new product called Eddy Link Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                Can you generate a product launch blog post I can use on our website?"""
   ):
    print(event)

Exception in thread Thread-604 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 158, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/opt/conda/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "uvloop/loop.pyx", line 1518, in uvloop.loop.Loop.run_until_complete
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 146, in _invoke_run_async
    async for event in self.run_async(
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 202, in run_async
    in

In [221]:
pip install --quiet gradio

Note: you may need to restart the kernel to use updated packages.


In [321]:
import requests
import gradio as gr
from google.auth.transport.requests import Request
import google.auth
import json

# Replace this with your Vertex AI agent stream URL
AGENT_STREAM_URL = "https://us-central1-aiplatform.googleapis.com/v1/projects/content-creation-agent-468223/locations/us-central1/reasoningEngines/1641201424362635264:streamQuery?alt=sse"

def query_agent(user_input):
    # Get default credentials
    credentials, _ = google.auth.default()
    
    # Refresh if token is expired or missing
    if not credentials.valid or credentials.expired:
        credentials.refresh(Request())

    # Get access token
    auth_token = credentials.token

    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }

    body = {
      "class_method":"stream_query",
        "input":{
            "user_id":"user_123",
            "message": user_input
        }
    }

    # Send POST request to stream endpoint
    response = requests.post(AGENT_STREAM_URL, headers=headers, json=body, stream=True)
    
    result_text = ""
    
    if(response.status_code == 200):

        #response.iter_lines():
        for line in response.iter_lines():
            if line:
                try:
                    decoded = line.decode("utf-8").replace("data: ", "")
                    chunk = json.loads(decoded)

                    # Correct: Accessing 'content' directly
                    content = chunk.get("content", {})
                    parts = content.get("parts", [])
                    for part in parts:
                        result_text += part.get("text", "")

                except Exception as e:
                    print("[!] Error parsing line:", e)
                    continue
    else:
        result_text = response.text
        
    return result_text


In [323]:
sample_prompt="""I want to announce a new product called VisionLink Pro.
It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration. 
The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video. 
The target audience is remote teams, global companies, and project managers.

OR

We’re launching a new smart home device called EcoGlow.
It’s a voice-controlled LED light that adjusts based on mood, time of day, and room activity.
It works with Google Assistant and Alexa, and is designed for energy efficiency.
The target audience is eco-conscious homeowners and tech-savvy renters.
"""

def query_agent_with_word_count(user_input):
    result = query_agent(user_input)
    word_count = len(result.split())
    return result, f"Words: {word_count}"

with gr.Blocks() as demo:
    gr.Markdown("### Ask Eddy's Blog Post Agent")
    with gr.Row():
        user_input = gr.Textbox(placeholder="Enter your question here...", lines=2, label="Your Prompt")
        submit_btn = gr.Button("Send")
    with gr.Column():
        output = gr.Textbox(lines=15, label="Agent Response")
        word_counter = gr.Textbox(label="Word Count", interactive=False, show_label=False, max_lines=1)
    with gr.Column():
        gr.Markdown("#### 🧪 Try This Sample Prompt")
        sample = gr.Textbox( value=sample_prompt, label="Example Prompt", lines=7, interactive=True)
    
    submit_btn.click(fn=query_agent_with_word_count, inputs=user_input, outputs=[output, word_counter])

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7884
* Running on public URL: https://b29ff66be54e1cb0c7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [324]:
demo.close()

Closing server running on port: 7884


In [326]:
remote_agent.delete(force=True)

AttributeError: 'AdkApp' object has no attribute 'delete'